In [2]:
import os
import csv
import random
from PIL import Image

# === Config ===
input_dir = "test_images"  # Thư mục chứa ảnh gốc
output_path = "input_data"
output_image_dir = os.path.join(output_path, "X_test")
csv_output_path = os.path.join(output_path, "Y_test.csv")

ROWS = 3
COLS = 5

# === Tạo thư mục output ===
os.makedirs(output_image_dir, exist_ok=True)

def slice_image(image_path, rows, cols):
    """Cắt ảnh thành rows x cols và trả về list các mảnh."""
    img = Image.open(image_path)
    width, height = img.size
    piece_width = width // cols
    piece_height = height // rows

    pieces = []
    for r in range(rows):
        for c in range(cols):
            left = c * piece_width
            upper = r * piece_height
            right = left + piece_width
            lower = upper + piece_height
            piece = img.crop((left, upper, right, lower))
            pieces.append(piece)
    return pieces

def create_shuffled_image(pieces, rows, cols, output_filename):
    """Tạo ảnh xáo trộn từ danh sách pieces và lưu lại."""
    piece_width, piece_height = pieces[0].size
    new_img = Image.new("RGB", (piece_width * cols, piece_height * rows))

    for idx, piece in enumerate(pieces):
        r = idx // cols
        c = idx % cols
        new_img.paste(piece, (c * piece_width, r * piece_height))

    new_img.save(output_filename)

def process_single_image(image_path, writer):
    """Xử lý 1 ảnh: cắt, xáo trộn, lưu ảnh và ghi mapping vào CSV."""
    pieces = slice_image(image_path, ROWS, COLS)

    # Tạo danh sách index [0..14] rồi shuffle
    original_indices = list(range(len(pieces)))
    shuffled_indices = original_indices.copy()

    # Đảm bảo shuffle không ra 0,1,2,...
    while True:
        random.shuffle(shuffled_indices)
        if shuffled_indices != original_indices:
            break

    # Sắp xếp mảnh theo thứ tự shuffled_indices
    shuffled_pieces = [pieces[i] for i in shuffled_indices]

    # Lưu ảnh xáo trộn
    base_name = os.path.basename(image_path)
    shuffled_image_name = os.path.splitext(base_name)[0] + "_shuffled.png"
    output_image_path = os.path.join(output_image_dir, shuffled_image_name)
    create_shuffled_image(shuffled_pieces, ROWS, COLS, output_image_path)

    # Ghi thông tin mapping vào CSV
    writer.writerow([shuffled_image_name] + shuffled_indices)

def generate_dataset():
    # Mở CSV và ghi header
    with open(csv_output_path, mode="w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        
        # Header
        header = ["image_filename"]
        for r in range(ROWS):
            for c in range(COLS):
                header.append(f"piece_at_{r}_{c}")
        writer.writerow(header)

        # Duyệt toàn bộ ảnh trong thư mục test_images
        for filename in os.listdir(input_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_path = os.path.join(input_dir, filename)
                print(f"Processing {filename}...")
                process_single_image(image_path, writer)

    print(f"\nDataset generated successfully!")
    print(f"- X_test images saved to: {output_image_dir}")
    print(f"- CSV file saved to: {csv_output_path}")

if __name__ == "__main__":
    generate_dataset()


Processing 10452_sat_08.jpg...
Processing 10452_sat_18.jpg...
Processing 111335_sat_00.jpg...
Processing 111335_sat_01.jpg...
Processing 111335_sat_02.jpg...
Processing 111335_sat_03.jpg...
Processing 111335_sat_04.jpg...
Processing 111335_sat_07.jpg...
Processing 111335_sat_08.jpg...
Processing 111335_sat_10.jpg...
Processing 111335_sat_12.jpg...
Processing 111335_sat_13.jpg...
Processing 111335_sat_15.jpg...
Processing 111335_sat_23.jpg...
Processing 111335_sat_24.jpg...
Processing 111335_sat_25.jpg...
Processing 111335_sat_26.jpg...
Processing 111335_sat_27.jpg...
Processing 111335_sat_35.jpg...
Processing 111335_sat_36.jpg...
Processing 111335_sat_37.jpg...
Processing 111335_sat_44.jpg...
Processing 111335_sat_45.jpg...
Processing 111335_sat_46.jpg...
Processing 111335_sat_47.jpg...
Processing 111335_sat_48.jpg...
Processing 111335_sat_54.jpg...
Processing 111335_sat_55.jpg...
Processing 111335_sat_56.jpg...
Processing 111335_sat_57.jpg...
Processing 111335_sat_58.jpg...
Processing